# Análise da Relação entre PIB Municipal e Emissões de GEE

Trabalho de Big Data – análise em nível **município × quinquênio** usando DuckDB e Python.

## 1. Imports e conexão com o DuckDB

In [ ]:
#!pip install duckdb
#!pip install statsmodels

import duckdb
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Conexão com DuckDB em memória
conn = duckdb.connect(database=':memory:')

## 2. Carregamento das bases de emissões e PIB

In [ ]:
# ============================================================
# 2. CARREGAMENTO DAS BASES
# ============================================================

# 2.1 Emissões de GEE por município e quinquênio
PARQUET_PATH = Path("emissoes_por_quinquenio_2002_2021.parquet")
conn.execute("""
    CREATE TABLE emissoes AS
    SELECT * FROM read_parquet(?)
""", [str(PARQUET_PATH)])

print("✅ Base de emissões carregada")
display(conn.execute("""
    SELECT quinquenio,
           COUNT(*) AS registros,
           SUM(emissao_t) AS emissao_total
    FROM emissoes
    GROUP BY quinquenio
    ORDER BY quinquenio
""").df())

# 2.2 PIB Municipal (PIB_M.csv)
PIB_PATH = Path("PIB_M.csv")
conn.execute("""
    CREATE TABLE pib_raw AS
    SELECT * FROM read_csv_auto(?)
""", [str(PIB_PATH)])

print("✅ Base de PIB bruto carregada")
display(conn.execute("""
    SELECT 
        MIN(ano) AS ano_min,
        MAX(ano) AS ano_max,
        COUNT(DISTINCT id_municipio) AS municipios,
        COUNT(*) AS registros
    FROM pib_raw
""").df())

## 3. Tratamento e agregação do PIB por município e quinquênio

In [ ]:
# ============================================================
# 3. TRATAMENTO DO PIB: CÓDIGO DO MUNICÍPIO + QUINQUÊNIOS
# ============================================================

# 3.1 Cria tabela PIB com código de município e quinquênio
conn.execute("""
    CREATE TABLE pib AS
    SELECT 
        SUBSTRING(id_municipio::VARCHAR, 1, 6) AS cod_municipio,
        ano,
        CASE 
            WHEN ano BETWEEN 2002 AND 2006 THEN '2002-2006'
            WHEN ano BETWEEN 2007 AND 2011 THEN '2007-2011'
            WHEN ano BETWEEN 2012 AND 2016 THEN '2012-2016'
            WHEN ano BETWEEN 2017 AND 2021 THEN '2017-2021'
            ELSE NULL
        END AS quinquenio,
        pib,
        impostos_liquidos,
        va,
        va_agropecuaria,
        va_industria,
        va_servicos,
        va_adespss
    FROM pib_raw
    WHERE ano BETWEEN 2002 AND 2021
""")

print("✅ Tabela PIB criada com quinquênios")
display(conn.execute("""
    SELECT quinquenio, COUNT(*) AS registros
    FROM pib
    GROUP BY quinquenio
    ORDER BY quinquenio
""").df())

# 3.2 Agrega PIB por município + quinquênio (nível certo de análise)
conn.execute("""
    CREATE TABLE pib_agregado AS
    SELECT 
        cod_municipio,
        quinquenio,
        SUM(pib) AS pib_total,
        SUM(impostos_liquidos) AS impostos_total,
        SUM(va) AS va_total,
        SUM(va_agropecuaria) AS va_agro_total,
        SUM(va_industria) AS va_ind_total,
        SUM(va_servicos) AS va_serv_total,
        SUM(va_adespss) AS va_adesp_total,
        COUNT(*) AS anos_no_quinquenio
    FROM pib
    WHERE quinquenio IS NOT NULL
    GROUP BY cod_municipio, quinquenio
""")

print("✅ PIB agregado por município e quinquênio")
display(conn.execute("SELECT * FROM pib_agregado LIMIT 10").df())

## 4. Tabela de municípios (de-para código IBGE)

In [ ]:
# ============================================================
# 4. TABELA DE MUNICÍPIOS (DE-PARA CÓDIGO IBGE)
# ============================================================

MUNICIPIOS_PATH = Path("municipios.csv")
df_municipios = pd.read_csv(MUNICIPIOS_PATH, encoding="utf-8")

df_municipios["cod_municipio"] = df_municipios["codigo_ibge"].astype(str).str[:6]
df_municipios["uf"] = df_municipios["uf"].astype(str)
df_municipios["cidade"] = df_municipios["nome"] + " (" + df_municipios["uf"] + ")"

# Registra no DuckDB
conn.register("municipios", df_municipios[["cod_municipio", "nome", "uf", "cidade"]])

print(f"✅ Tabela de municípios carregada. Total de municípios: {len(df_municipios)}")
display(conn.execute("SELECT * FROM municipios LIMIT 5").df())

## 5. Construção da base principal: município × quinquênio

In [ ]:
# ============================================================
# 5. EMISSÕES COM CÓDIGO DO MUNICÍPIO E BASE MUNICÍPIO × QUINQUÊNIO
# ============================================================

# 5.1 Junta emissões com tabela de municípios
conn.execute("""
    CREATE OR REPLACE TABLE emissoes_temp AS
    SELECT 
        e.*,
        m.cod_municipio
    FROM emissoes e
    JOIN municipios m 
      ON e.Cidade = m.cidade
""")

print("✅ Tabela emissoes_temp criada (emissões + cod_municipio)")
display(conn.execute("SELECT * FROM emissoes_temp LIMIT 5").df())

# 5.2 Base principal: município × quinquênio (emissões + PIB)
conn.execute("""
    CREATE OR REPLACE TABLE base_mun_quinquenio AS
    SELECT
        et.cod_municipio,
        et.Cidade,
        et.quinquenio,
        SUM(et.emissao_t) AS emissao_total_quinquenio,
        pa.pib_total       AS pib_total_quinquenio,
        pa.impostos_total,
        pa.va_total,
        pa.va_agro_total,
        pa.va_ind_total,
        pa.va_serv_total,
        pa.va_adesp_total,
        pa.anos_no_quinquenio
    FROM emissoes_temp et
    JOIN pib_agregado pa
      ON et.cod_municipio = pa.cod_municipio
     AND et.quinquenio   = pa.quinquenio
    GROUP BY
        et.cod_municipio,
        et.Cidade,
        et.quinquenio,
        pa.pib_total,
        pa.impostos_total,
        pa.va_total,
        pa.va_agro_total,
        pa.va_ind_total,
        pa.va_serv_total,
        pa.va_adesp_total,
        pa.anos_no_quinquenio
""")

print("✅ Tabela base_mun_quinquenio criada (nível município × quinquênio)")
display(conn.execute("SELECT * FROM base_mun_quinquenio LIMIT 5").df())

## 6. Visualização: gráfico de dispersão PIB × emissões

In [ ]:
# ============================================================
# 6. VISUALIZAÇÃO: GRÁFICO DE DISPERSÃO PIB × EMISSÕES
# ============================================================

df_base = conn.execute("""
    SELECT pib_total_quinquenio, emissao_total_quinquenio, quinquenio
    FROM base_mun_quinquenio
""").df()

plt.figure()
for q in sorted(df_base["quinquenio"].unique()):
    sub = df_base[df_base["quinquenio"] == q]
    plt.scatter(sub["pib_total_quinquenio"], sub["emissao_total_quinquenio"], alpha=0.4, label=q)

plt.xlabel("PIB total no quinquênio")
plt.ylabel("Emissões totais de GEE no quinquênio")
plt.title("PIB × Emissões – municípios por quinquênio")
plt.legend()
plt.tight_layout()
plt.show()

## 7. Relação PIB × emissões: correlações e significância

In [ ]:
# ============================================================
# 7. RELAÇÃO PIB × EMISSÕES: CORRELAÇÕES E SIGNIFICÂNCIA
# ============================================================

print("\n💡 Correlação por quinquênio (PIB municipal vs emissões totais de GEE)")

# 7.1 Correlação por quinquênio
display(conn.execute("""
    SELECT 
        quinquenio,
        COUNT(DISTINCT cod_municipio) AS num_municipios,
        SUM(pib_total_quinquenio)     AS pib_total,
        SUM(emissao_total_quinquenio) AS emissao_total,
        ROUND(AVG(pib_total_quinquenio), 2)      AS pib_medio,
        ROUND(AVG(emissao_total_quinquenio), 2)  AS emissao_media,
        ROUND(CORR(pib_total_quinquenio, emissao_total_quinquenio), 4) AS correlacao_pearson
    FROM base_mun_quinquenio
    GROUP BY quinquenio
    ORDER BY quinquenio
""").df())

# 7.2 Correlação geral (todos os municípios e quinquênios)
print("\n💡 Correlação geral (todos os municípios e períodos)")

display(conn.execute("""
    WITH stats AS (
        SELECT 
            CORR(pib_total_quinquenio, emissao_total_quinquenio) AS correlacao_geral,
            COUNT(*) AS total_observacoes,
            COUNT(DISTINCT cod_municipio) AS total_municipios,
            SUM(pib_total_quinquenio) AS pib_total,
            SUM(emissao_total_quinquenio) AS emissao_total
        FROM base_mun_quinquenio
    )
    SELECT 
        ROUND(correlacao_geral, 4) AS correlacao_pearson,
        total_observacoes,
        total_municipios,
        ROUND(pib_total, 2) AS pib_total,
        ROUND(emissao_total, 2) AS emissao_total,
        CASE 
            WHEN correlacao_geral > 0.7 THEN 'HIPÓTESE FORTEMENTE CONFIRMADA'
            WHEN correlacao_geral > 0.4 THEN 'HIPÓTESE CONFIRMADA (correlação moderada)'
            WHEN correlacao_geral > 0.2 THEN 'HIPÓTESE PARCIALMENTE CONFIRMADA (correlação fraca)'
            ELSE 'HIPÓTESE NÃO CONFIRMADA'
        END AS conclusao
    FROM stats
""").df())

# 7.3 Teste de significância da correlação por quinquênio (t-statistic)
print("\n💡 Teste de significância (t-statistic aproximado) por quinquênio")
print("   Interpretação: p < 0.05 indica que a relação dificilmente é ao acaso")

display(conn.execute("""
    WITH stats_quinquenio AS (
        SELECT 
            quinquenio,
            CORR(pib_total_quinquenio, emissao_total_quinquenio) AS correlacao,
            COUNT(*) AS n,
            CORR(pib_total_quinquenio, emissao_total_quinquenio)
                * SQRT( (COUNT(*) - 2) / (1 - POWER(CORR(pib_total_quinquenio, emissao_total_quinquenio), 2)) )
              AS t_statistic
        FROM base_mun_quinquenio
        GROUP BY quinquenio
    )
    SELECT 
        quinquenio,
        ROUND(correlacao, 4) AS correlacao,
        n AS tamanho_amostra,
        ROUND(t_statistic, 4) AS t_statistic_aproximado,
        CASE 
            WHEN ABS(t_statistic) > 2.576 THEN 'p < 0.01 (Muito significativo)'
            WHEN ABS(t_statistic) > 1.960 THEN 'p < 0.05 (Significativo)'
            WHEN ABS(t_statistic) > 1.645 THEN 'p < 0.10 (Marginalmente significativo)'
            ELSE 'Não significativo'
        END AS significancia_estatistica
    FROM stats_quinquenio
    ORDER BY quinquenio
""").df())

## 8. Regressão linear simples (OLS) entre PIB e emissões

In [ ]:
# ============================================================
# 8. REGRESSÃO LINEAR SIMPLES: EMISSÕES ~ PIB
# ============================================================

df_reg = conn.execute("""
    SELECT 
        pib_total_quinquenio AS pib,
        emissao_total_quinquenio AS emis
    FROM base_mun_quinquenio
    WHERE pib_total_quinquenio > 0
      AND emissao_total_quinquenio > 0
""").df()

X = sm.add_constant(df_reg["pib"])
y = df_reg["emis"]

model = sm.OLS(y, X).fit()
print(model.summary())

## 9. Análise por faixas de PIB

In [ ]:
# ============================================================
# 9. ANÁLISE POR FAIXAS DE PIB
# ============================================================

print("\n💡 Municípios com PIB maior deveriam ter emissões médias maiores")

display(conn.execute("""
    SELECT 
        CASE 
            WHEN pib_total_quinquenio < 100000    THEN '1. Até 100 mil'
            WHEN pib_total_quinquenio < 500000   THEN '2. 100-500 mil'
            WHEN pib_total_quinquenio < 1000000  THEN '3. 500mil-1M'
            WHEN pib_total_quinquenio < 5000000  THEN '4. 1M-5M'
            ELSE '5. Acima de 5M'
        END AS faixa_pib,
        COUNT(*) AS num_registros,
        ROUND(AVG(pib_total_quinquenio), 2)      AS pib_medio,
        ROUND(AVG(emissao_total_quinquenio), 2)  AS emissao_media,
        ROUND(MIN(emissao_total_quinquenio), 2)  AS emissao_minima,
        ROUND(MAX(emissao_total_quinquenio), 2)  AS emissao_maxima
    FROM base_mun_quinquenio
    GROUP BY faixa_pib
    ORDER BY faixa_pib
""").df())

## 10. Crescimento do PIB e das emissões ao longo do tempo

In [ ]:
# ============================================================
# 10. CRESCIMENTO DO PIB vs CRESCIMENTO DAS EMISSÕES
# ============================================================

print("\n💡 Crescimento do PIB vs crescimento das emissões por município e quinquênio")

display(conn.execute("""
    WITH crescimento AS (
        SELECT 
            cod_municipio,
            Cidade,
            quinquenio,
            pib_total_quinquenio       AS pib,
            emissao_total_quinquenio   AS emissao,
            LAG(pib_total_quinquenio) 
                OVER (PARTITION BY cod_municipio ORDER BY quinquenio) AS pib_anterior,
            LAG(emissao_total_quinquenio) 
                OVER (PARTITION BY cod_municipio ORDER BY quinquenio) AS emissao_anterior
        FROM base_mun_quinquenio
    )
    SELECT 
        quinquenio,
        COUNT(*) AS num_municipios,
        ROUND(AVG( (pib - pib_anterior) / NULLIF(pib_anterior, 0) * 100 ), 2)      AS crescimento_pib_pct_medio,
        ROUND(AVG( (emissao - emissao_anterior) / NULLIF(emissao_anterior, 0) * 100 ), 2) AS crescimento_emissao_pct_medio,
        ROUND(CORR(
            (pib - pib_anterior) / NULLIF(pib_anterior, 0), 
            (emissao - emissao_anterior) / NULLIF(emissao_anterior, 0)
        ), 4) AS correlacao_crescimentos
    FROM crescimento
    WHERE pib_anterior IS NOT NULL
      AND emissao_anterior IS NOT NULL
    GROUP BY quinquenio
    ORDER BY quinquenio
""").df())

print("\n💡 Municípios com maior aumento de emissões entre 2002-2006 e 2017-2021")

display(conn.execute("""
    WITH emissoes_periodo AS (
        SELECT 
            cod_municipio,
            Cidade,
            MIN(CASE WHEN quinquenio = '2002-2006' THEN emissao_total_quinquenio END) AS emissao_2002,
            MAX(CASE WHEN quinquenio = '2017-2021' THEN emissao_total_quinquenio END) AS emissao_2021,
            MIN(CASE WHEN quinquenio = '2002-2006' THEN pib_total_quinquenio END)    AS pib_2002,
            MAX(CASE WHEN quinquenio = '2017-2021' THEN pib_total_quinquenio END)    AS pib_2021
        FROM base_mun_quinquenio
        GROUP BY cod_municipio, Cidade
        HAVING emissao_2002 IS NOT NULL 
           AND emissao_2021 IS NOT NULL
           AND emissao_2002 > 0
    )
    SELECT 
        Cidade,
        ROUND(emissao_2002, 2) AS emissao_2002_2006,
        ROUND(emissao_2021, 2) AS emissao_2017_2021,
        ROUND(emissao_2021 - emissao_2002, 2) AS crescimento_absoluto,
        ROUND((emissao_2021 - emissao_2002) / NULLIF(emissao_2002, 0) * 100, 2) AS crescimento_emissao_pct,
        ROUND(pib_2021 - pib_2002, 2) AS crescimento_pib_absoluto,
        ROUND((pib_2021 - pib_2002) / NULLIF(pib_2002, 0) * 100, 2) AS crescimento_pib_pct
    FROM emissoes_periodo
    ORDER BY crescimento_absoluto DESC
    LIMIT 30
""").df())

## 11. Intensidade de carbono e evolução temporal de emissões, PIB e intensidade

In [ ]:
# ============================================================
# 11. INTENSIDADE DE CARBONO (EMISSÕES / PIB) E SÉRIES TEMPORAIS
# ============================================================

print("\n💡 Intensidade de carbono por quinquênio (emissões / PIB)")

df_int = conn.execute("""
    SELECT 
        quinquenio,
        SUM(emissao_total_quinquenio) AS emis,
        SUM(pib_total_quinquenio) AS pib,
        SUM(emissao_total_quinquenio) / SUM(pib_total_quinquenio) AS intensidade
    FROM base_mun_quinquenio
    WHERE pib_total_quinquenio > 0
    GROUP BY quinquenio
    ORDER BY quinquenio
""").df()

display(df_int)

# Gráfico de emissões totais
plt.figure()
plt.plot(df_int["quinquenio"], df_int["emis"], marker="o")
plt.xlabel("Quinquênio")
plt.ylabel("Emissões totais de GEE")
plt.title("Emissões totais por quinquênio")
plt.tight_layout()
plt.show()

# Gráfico de PIB total
plt.figure()
plt.plot(df_int["quinquenio"], df_int["pib"], marker="o")
plt.xlabel("Quinquênio")
plt.ylabel("PIB total")
plt.title("PIB total por quinquênio")
plt.tight_layout()
plt.show()

# Gráfico de intensidade de carbono
plt.figure()
plt.plot(df_int["quinquenio"], df_int["intensidade"], marker="o")
plt.xlabel("Quinquênio")
plt.ylabel("Intensidade de carbono (emissões / PIB)")
plt.title("Intensidade de carbono por quinquênio")
plt.tight_layout()
plt.show()

## 12. Análises setoriais de emissões

In [ ]:
# ============================================================
# 12. ANÁLISES SETORIAIS DE EMISSÕES (SEM DUPLICAR PIB)
# ============================================================

print("\n💡 Crescimento de emissões por setor (2002-2006 vs 2017-2021)")

display(conn.execute("""
    WITH emissoes_setor AS (
        SELECT 
            Setor_Agrupado,
            SUM(CASE WHEN quinquenio = '2002-2006' THEN emissao_t END) AS emissao_2002,
            SUM(CASE WHEN quinquenio = '2017-2021' THEN emissao_t END) AS emissao_2021
        FROM emissoes_temp
        GROUP BY Setor_Agrupado
    )
    SELECT 
        Setor_Agrupado,
        ROUND(emissao_2002, 2) AS emissao_2002_2006,
        ROUND(emissao_2021, 2) AS emissao_2017_2021,
        ROUND(emissao_2021 - emissao_2002, 2) AS variacao_absoluta,
        ROUND((emissao_2021 - emissao_2002) / NULLIF(emissao_2002, 0) * 100, 2) AS crescimento_pct,
        CASE 
            WHEN emissao_2021 > emissao_2002 THEN 'Crescimento'
            WHEN emissao_2021 < emissao_2002 THEN 'Redução'
            ELSE 'Estável'
        END AS status
    FROM emissoes_setor
    ORDER BY crescimento_pct DESC
""").df())

print("\n💡 Tendência temporal das emissões por setor")

display(conn.execute("""
    SELECT 
        Setor_Agrupado,
        quinquenio,
        ROUND(SUM(emissao_t), 2) AS emissao_total,
        COUNT(DISTINCT cod_municipio) AS num_municipios,
        ROUND(AVG(emissao_t), 2) AS emissao_media_municipio
    FROM emissoes_temp
    GROUP BY Setor_Agrupado, quinquenio
    ORDER BY Setor_Agrupado, quinquenio
""").df())

print("\n💡 Participação setorial nas emissões totais de cada quinquênio")

display(conn.execute("""
    WITH emissoes_totais AS (
        SELECT quinquenio, SUM(emissao_t) AS total_geral
        FROM emissoes_temp
        GROUP BY quinquenio
    )
    SELECT 
        e.quinquenio,
        e.Setor_Agrupado,
        ROUND(SUM(e.emissao_t), 2) AS emissao_setor,
        ROUND(t.total_geral, 2) AS emissao_total,
        ROUND(SUM(e.emissao_t) / t.total_geral * 100, 2) AS participacao_pct
    FROM emissoes_temp e
    JOIN emissoes_totais t ON e.quinquenio = t.quinquenio
    GROUP BY e.quinquenio, e.Setor_Agrupado, t.total_geral
    ORDER BY e.quinquenio, participacao_pct DESC
""").df())

print("\n💡 CAGR das emissões por setor (crescimento anual composto)")

display(conn.execute("""
    WITH setor_periodo AS (
        SELECT 
            Setor_Agrupado,
            SUM(CASE WHEN quinquenio = '2002-2006' THEN emissao_t END) AS emissao_inicial,
            SUM(CASE WHEN quinquenio = '2017-2021' THEN emissao_t END) AS emissao_final,
            15 AS anos -- aprox. 2003 → 2018
        FROM emissoes_temp
        GROUP BY Setor_Agrupado
        HAVING SUM(CASE WHEN quinquenio = '2002-2006' THEN emissao_t END) > 0
    )
    SELECT 
        Setor_Agrupado,
        ROUND(emissao_inicial, 2) AS emissao_inicial,
        ROUND(emissao_final, 2) AS emissao_final,
        ROUND((POWER(emissao_final / NULLIF(emissao_inicial, 0), 1.0/anos) - 1) * 100, 2) AS cagr_pct,
        CASE 
            WHEN (POWER(emissao_final / NULLIF(emissao_inicial, 0), 1.0/anos) - 1) * 100 > 5  THEN 'Alto crescimento'
            WHEN (POWER(emissao_final / NULLIF(emissao_inicial, 0), 1.0/anos) - 1) * 100 > 0  THEN 'Crescimento moderado'
            WHEN (POWER(emissao_final / NULLIF(emissao_inicial, 0), 1.0/anos) - 1) * 100 > -5 THEN 'Estável'
            ELSE 'Em redução'
        END AS classificacao
    FROM setor_periodo
    ORDER BY cagr_pct DESC
""").df())

print("\n💡 Setores com redução relevante de emissões (2002-2006 → 2017-2021)")

display(conn.execute("""
    WITH reducao_setor AS (
        SELECT 
            Setor_Agrupado,
            SUM(CASE WHEN quinquenio = '2002-2006' THEN emissao_t END) AS emissao_inicial,
            SUM(CASE WHEN quinquenio = '2017-2021' THEN emissao_t END) AS emissao_final
        FROM emissoes_temp
        GROUP BY Setor_Agrupado
        HAVING SUM(CASE WHEN quinquenio = '2017-2021' THEN emissao_t END) <
               SUM(CASE WHEN quinquenio = '2002-2006' THEN emissao_t END)
    )
    SELECT 
        Setor_Agrupado,
        ROUND(emissao_inicial, 2) AS emissao_2002_2006,
        ROUND(emissao_final, 2) AS emissao_2017_2021,
        ROUND(emissao_inicial - emissao_final, 2) AS reducao_absoluta,
        ROUND((emissao_inicial - emissao_final) / NULLIF(emissao_inicial, 0) * 100, 2) AS reducao_percentual
    FROM reducao_setor
    ORDER BY reducao_percentual DESC
""").df())

print("\n💡 Cases de sucesso: municípios/setores com forte redução de emissões")

display(conn.execute("""
    WITH reducao_municipio_setor AS (
        SELECT 
            Cidade,
            Setor_Agrupado,
            MIN(CASE WHEN quinquenio = '2002-2006' THEN emissao_t END) AS emissao_inicial,
            MAX(CASE WHEN quinquenio = '2017-2021' THEN emissao_t END) AS emissao_final
        FROM emissoes_temp
        GROUP BY Cidade, Setor_Agrupado
        HAVING MIN(CASE WHEN quinquenio = '2002-2006' THEN emissao_t END) >
               MAX(CASE WHEN quinquenio = '2017-2021' THEN emissao_t END)
           AND MIN(CASE WHEN quinquenio = '2002-2006' THEN emissao_t END) > 1000
    )
    SELECT 
        Setor_Agrupado,
        Cidade,
        ROUND(emissao_inicial, 2) AS emissao_inicial,
        ROUND(emissao_final, 2)   AS emissao_final,
        ROUND((emissao_inicial - emissao_final) / emissao_inicial * 100, 2) AS reducao_pct
    FROM reducao_municipio_setor
    ORDER BY Setor_Agrupado, reducao_pct DESC
    LIMIT 50
""").df())

## 13. Desempenho de macro-setores: emissões vs valor adicionado

In [ ]:
# ============================================================
# 13. DESEMPENHO DE MACRO-SETOR: EMISSÕES vs VALOR ADICIONADO
# ============================================================

print("\n💡 Desempenho de macro-setores (Agro / Indústria / Serviços): emissões vs VA")

display(conn.execute("""
    -- Emissões por macro-setor (Agro / Indústria / Serviços) e quinquênio
    WITH emissao_macro AS (
        SELECT 
            CASE 
                WHEN Setor_Agrupado ILIKE '%agro%' OR Setor_Agrupado ILIKE '%agr%' 
                    THEN 'Agro'
                WHEN Setor_Agrupado ILIKE '%ind%' 
                    THEN 'Indústria'
                WHEN Setor_Agrupado ILIKE '%serv%' 
                    THEN 'Serviços'
                ELSE 'Outros'
            END AS macro_setor,
            quinquenio,
            SUM(emissao_t) AS emissao_total
        FROM emissoes_temp
        GROUP BY macro_setor, quinquenio
    ),
    -- Valor adicionado setorial agregado por quinquênio (a partir do PIB)
    va_macro_long AS (
        SELECT 'Agro'      AS macro_setor, quinquenio, SUM(va_agro_total) AS va_total
        FROM pib_agregado
        GROUP BY quinquenio
        UNION ALL
        SELECT 'Indústria' AS macro_setor, quinquenio, SUM(va_ind_total) AS va_total
        FROM pib_agregado
        GROUP BY quinquenio
        UNION ALL
        SELECT 'Serviços'  AS macro_setor, quinquenio, SUM(va_serv_total) AS va_total
        FROM pib_agregado
        GROUP BY quinquenio
    ),
    desempenho AS (
        SELECT 
            e.macro_setor,
            e.quinquenio,
            e.emissao_total,
            v.va_total
        FROM emissao_macro e
        JOIN va_macro_long v
          ON e.macro_setor = v.macro_setor
         AND e.quinquenio = v.quinquenio
        WHERE e.macro_setor IN ('Agro','Indústria','Serviços')
    ),
    resumo AS (
        SELECT 
            macro_setor,
            SUM(CASE WHEN quinquenio = '2002-2006' THEN emissao_total END) AS emissao_inicial,
            SUM(CASE WHEN quinquenio = '2017-2021' THEN emissao_total END) AS emissao_final,
            SUM(CASE WHEN quinquenio = '2002-2006' THEN va_total END)      AS va_inicial,
            SUM(CASE WHEN quinquenio = '2017-2021' THEN va_total END)      AS va_final
        FROM desempenho
        GROUP BY macro_setor
    )
    SELECT 
        macro_setor,
        ROUND(emissao_inicial, 2) AS emissao_2002_2006,
        ROUND(emissao_final, 2)   AS emissao_2017_2021,
        ROUND((emissao_final - emissao_inicial) / NULLIF(emissao_inicial, 0) * 100, 2) AS variacao_emissao_pct,
        ROUND(va_inicial, 2)      AS va_2002_2006,
        ROUND(va_final, 2)        AS va_2017_2021,
        ROUND((va_final - va_inicial) / NULLIF(va_inicial, 0) * 100, 2) AS variacao_va_pct,
        CASE 
            WHEN (emissao_final - emissao_inicial) < 0 AND (va_final - va_inicial) > 0 
                THEN 'Cresceu com redução de emissões'
            WHEN (emissao_final - emissao_inicial) < (va_final - va_inicial) 
                THEN 'Crescimento mais limpo'
            WHEN (emissao_final - emissao_inicial) > (va_final - va_inicial) 
                THEN 'Emissões cresceram mais que economia'
            ELSE 'Performance neutra'
        END AS desempenho_sustentavel
    FROM resumo
    WHERE va_inicial > 0 AND emissao_inicial > 0
    ORDER BY macro_setor
""").df())

print("\n🎯 Fim da análise. A hipótese foi testada em nível município × quinquênio,")